<a href="https://colab.research.google.com/github/SaatvikP/POS_Tagging_using_Word_Embedding/blob/main/POS_Tagging_SVM_LOGREG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import string
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack

In [ ]:
ds = load_dataset("batterydata/pos_tagging")
train_data = pd.DataFrame(ds["train"])
test_data = pd.DataFrame(ds["test"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['words', 'labels'],
        num_rows: 13054
    })
    test: Dataset({
        features: ['words', 'labels'],
        num_rows: 1451
    })
})

In [ ]:
train_sentences = train_data["words"][:1000]
train_tags = train_data["labels"][:1000]

test_sentences = test_data["words"]
test_tags = test_data["labels"]

In [ ]:
train_words = [word for sentence in train_sentences for word in sentence]
train_labels = [tag for tags in train_tags for tag in tags]

test_words = [word for sentence in test_sentences for word in sentence]
test_labels = [tag for tags in test_tags for tag in tags]

In [ ]:
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
X_train_tfidf = vectorizer.fit_transform(train_words)
X_test_tfidf = vectorizer.transform(test_words)

In [ ]:
def extract_features(words):
    is_uppercase = np.array([1 if word[0].isupper() else 0 for word in words]).reshape(-1, 1)
    is_punctuation = np.array([1 if word in string.punctuation else 0 for word in words]).reshape(-1, 1)
    return is_uppercase, is_punctuation

train_uppercase, train_punctuation = extract_features(train_words)
test_uppercase, test_punctuation = extract_features(test_words)


In [ ]:
X_train = hstack([X_train_tfidf, train_uppercase, train_punctuation])
X_test = hstack([X_test_tfidf, test_uppercase, test_punctuation])

In [ ]:
all_labels = train_labels + test_labels
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

LabelEncoder()

In [ ]:
y_train = label_encoder.transform(train_labels)
y_test = label_encoder.transform(test_labels)

In [ ]:
log_reg = LogisticRegression(max_iter=5000, verbose=1)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=5000, verbose=1)

In [ ]:
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', verbose=1)
svm_model.fit(X_train, y_train)

[LibSVM]

SVC(verbose=1)

In [ ]:
y_pred_log = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_log)

print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7503


In [ ]:
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm:.4f}")

SVM Accuracy: 0.7767
